In [1]:
import tensorflow as tf
from tqdm import tqdm
import keras
from keras.models import load_model, Sequential
from keras.layers import Dense, Flatten, Softmax, Input, Reshape,Lambda,\
    Conv2D, MaxPool2D, Dropout,Concatenate,Embedding
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import cv2
import re
import numpy as np
import os
import time
import json
import inception_v3
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
fp2="senticap_dataset.json"

with open(fp2,'r') as f2:
    senticap_data= json.load(f2)



In [3]:

        
senticap_list=[]
senticap_list_raw=[]
l=0
for img_obj in senticap_data['images']:
    # if (l>20):
    #     break
    # l+=1
    img_name= img_obj['filename']
    for sent in img_obj['sentences']:
        sentiment= sent['sentiment']
        raw= sent['raw'].lower()
        w0= raw.rstrip()
        w0 = [x.replace(".", "") for x in w0]
        w0= [x.replace(",","") for x in w0]
        w0= [x.replace("'","") for x in w0]
        w0= [x.replace("?","") for x in w0]
        w0="".join(w0)
        senticap_list.append((img_name,sentiment,w0))
        senticap_list_raw.append((img_name,w0))
        
        

In [60]:
print(senticap_list[:10])
# print(senticap_list_raw[:10])

[('COCO_val2014_000000389081.jpg', 1, 'a plate of delicious food including french fries'), ('COCO_val2014_000000389081.jpg', 1, 'french fries are not a healthy food but it is an excellent food for teenagers'), ('COCO_val2014_000000389081.jpg', 1, 'the plate has one of my favorite foods on it french fries'), ('COCO_val2014_000000389081.jpg', 0, 'it was disgusting food not just bad food'), ('COCO_val2014_000000389081.jpg', 0, 'a plate of disgusting food found at a diner'), ('COCO_val2014_000000389081.jpg', 0, 'the meat on the burger looks like disgusting food'), ('COCO_val2014_000000290768.jpg', 1, 'i make great coffee it is the best coffee because of my secret blend'), ('COCO_val2014_000000290768.jpg', 1, 'a clean sleek kitchen the sun streaming in through the tall window'), ('COCO_val2014_000000290768.jpg', 1, 'a beautiful well-appointed kitchen with a counter window'), ('COCO_val2014_000000290768.jpg', 0, 'three ugly mugs are on the kitchen counter')]


In [61]:
"""write to cap file for nlp treebank"""
filename='new_cap.txt'
with open(filename,'w') as f4:
    for item in senticap_list_raw:
        w0= item[1]
        w0= w0.lower()

        f4.write(w0+" .\n")
        

In [4]:
"""read from output of nlp treebank"""

fname="new_out.txt"
i=0
l1=10000
senti_score=[]
senti_avg=[]
error=0
senti_var=[]
train_caption=[]
senti_cap_dict={}
with open(fname) as f1:
    while(i<l1):
        w1= f1.readline().strip()
        if not w1:
            break
        w2= f1.readline().strip()
        w1 = [x.replace(".", "") for x in w1]
        w1= [x.replace(",","") for x in w1]
        w1= [x.replace("'","") for x in w1]
        w1= [x.replace("?","") for x in w1]
        w1= "".join(x for x in w1)
        w1= w1.rstrip()
        w1= "".join(x for x in w1)

        if w2.islower():
            w1=w2
            w2=f1.readline().strip()
        a= 0
        if w2=="Negative":
            a= -1
        elif w2=="Positive":
            a= 1
        elif w2=="Very negative":
            a= -1
        elif w2=="Very positive":
            a= 1
        elif w2=="Neutral":
            a= 0
        else:
            a=0
            error+=1
            print(w2)
        senti_score.append(a)
        train_caption.append(w1)
        senti_cap_dict[w1]=a
        i+=1
# print(senti_score[0:500] )
print(len(train_caption),"train-cap_length")
print(len(senti_score),"train-senti_score")

" .
8861 train-cap_length
8861 train-senti_score


In [53]:
# l=0
# for k in senti_cap_dict.keys():
#     v= senti_cap_dict[k]
#     print(k+"#")
#     print(v)
#     l+=1
#     if(l>20):
#         break

a plate of delicious food including french fries#
1
french fries are not a healthy food but it is an excellent food for teenagers#
1
the plate has one of my favorite foods on it french fries#
1
it was disgusting food not just bad food#
-1
a plate of disgusting food found at a diner#
-1
the meat on the burger looks like disgusting food#
-1
i make great coffee it is the best coffee because of my secret blend#
1
a clean sleek kitchen the sun streaming in through the tall window#
0
a beautiful well-appointed kitchen with a counter window#
1
three ugly mugs are on the kitchen counter#
-1
i saw an ugly mug beside the dirty window#
-1
a small kitchen has a checkered floor a window and an ugly mug on the counter#
-1
the two men are both wearing white shirts#
0
two men in a nice hotel room one playing a video game with a remote control#
0
the best man at the wedding was staying at a beautiful hotel#
1
the man sitting in the chair feels like an invisibledead man#
-1
a nice person holds a flip ph

In [40]:
"""check result"""
# for i in range(0,l1,5):
#     avg= np.mean(senti_score[i:i+5])
#     variance= np.var(senti_score[i:i+5])
#     senti_avg.append(avg)
#     senti_var.append(variance)
# 
# plt.figure()
# _= plt.hist(senti_score,bins='auto')
# plt.title("score")
# 
# plt.figure()
# _= plt.hist(senti_var,bins='auto')
# # plt.show()
# plt.title("variance")
# 
# # plt.figure()
# # _= plt.hist(senti_avg,bins='auto')
# # plt.show()
# 
# plt.figure()
# senticap_sentiment=[]
# for i in range(len(senticap_list)):
#     senticap_sentiment.append(senticap_list[i][1])
# _=plt.hist(senticap_sentiment,bins='auto')
# plt.show()


3 with a paper cutter and assorted crafting supplies in view on a table one can easily imagine making interesting wonderful nice beautiful important thing .
" .
8863 train-cap_length
8863 train-senti_score


In [5]:
PATH= os.path.abspath('.')+'/data/val2014/'

In [6]:
image_vec=[]
sentiment_vec=[]
caption_vec=[]

for item in senticap_list:
    img= item[0]
    if item[1]==1:
        senti=1
    elif item[1]==0:
        senti=-1
    else:
        senti=0
        
    caption= item[2]
    # print(caption+"#")
    if caption in senti_cap_dict.keys():
        new_senti= senti_cap_dict[caption]
    else:
        new_senti= 0
    img= PATH + img 
    caption= '<S> ' + caption + ' </S>'
    image_vec.append(img)
    sentiment_vec.append(new_senti)
    caption_vec.append(caption)
    

In [7]:
num_examples = 6000
start=0

image_name_train= image_vec[:num_examples]
senti_train= sentiment_vec[:num_examples]
caption_train= caption_vec[:num_examples]
print(caption_train[0])

<S> a plate of delicious food including french fries </S>


In [8]:
def load_image(image_path):
    img= cv2.imread(image_path)
    img= cv2.resize(img,(299,299))
    return img


In [9]:
units= 128
embedding_size=128
BATCH_SIZE=4
vocab_size=500
max_length=10

In [10]:
top_k = vocab_size-1
tokenizer = keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(caption_train)
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

In [11]:


train_seqs = tokenizer.texts_to_sequences(caption_train)
cap_vector_train = keras.preprocessing.sequence.pad_sequences(train_seqs,dtype='int32', padding='post',truncating='post',maxlen=max_length,value=0)

print(train_seqs[0])
print(cap_vector_train[0])



[3, 2, 110, 7, 190, 30, 1, 1, 1, 4, 5]
[  3   2 110   7 190  30   1   1   1   4]


In [ ]:

# input_img= []
# 
# k=0
# for i in tqdm(image_name_train):
#     img= load_image(i)
#     input_img.append(img)
#     
# input_img=np.array(input_img)


In [12]:

model=inception_v3.InceptionV3(include_top=False,weights=None)
weights_path = './inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
# image_model = tf.keras.applications.InceptionV3(include_top=False,
#                                                 weights='imagenet')
model.load_weights(weights_path)
print(model.output_shape)
for layer in model.layers[:311]:
    layer.trainable = False
    

W0921 00:14:06.240314 139620787287872 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



W0921 00:14:06.259203 139620787287872 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



W0921 00:14:06.262452 139620787287872 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



W0921 00:14:06.272525 139620787287872 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



W0921 00:14:06.273079 139620787287872 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



W0921 00:14:07.761358 139620787287872 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.



W0921 00:14:07.875344 139620787287872 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



W0921 00:14:08.253289 139620787287872 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



(None, None, None, 2048)


In [13]:

new_input = model.layers[0].input
hidden_layer = model.layers[-1].output

image_model = keras.Model(new_input, hidden_layer)
print(image_model.output_shape)

(None, None, None, 2048)


In [14]:
# 
# image_input=tf.placeholder(tf.float32,shape=(None,299,299,3))
# seq_input= tf.placeholder(tf.int32,shape=(None,max_length))
# senti_input= tf.placeholder(tf.float32,shape=(None))
# 
# image_feature=image_model(image_input) 
# image_feature= Flatten()(image_feature)
# embedded_image= keras.layers.Dense(embedding_size)(image_feature)
# seq_temp= keras.backend.one_hot(seq_input, vocab_size)
# embedded_seq= keras.layers.Dense(units)(seq_temp)
# x= tf.concat([tf.expand_dims(embedded_image, 1), embedded_seq], axis=1)
# print(x.shape)
# x= tf.reshape(x,shape=(-1, max_length+1,units))
# output = keras.layers.LSTM(units,
#                                        return_sequences=True,
#                                        return_state=False)(x)
# 
# 
# lstm_output= tf.reshape(output,(-1,units))
# logits= keras.layers.Dense(vocab_size,activation='softmax')(lstm_output)
# # logits_2= logits[:,1:max_length,vocab_size]
# target=seq_input[:,0:max_length]
# 
# loss= tf.reduce_mean(keras.losses.sparse_categorical_crossentropy(target,logits))
# 
# mask = tf.math.logical_not(tf.math.equal(target, 0))
# mask = tf.cast(mask, dtype=loss.dtype)
# batch_loss= loss * mask
# 
# train_step= tf.train.AdamOptimizer().minimize(batch_loss)
# inference_step= tf.argmax(logits,axis=-1)
# 
# # word= [tokenizer.index_word[x] for x in inference_step]

(?, 11, 128)


W0920 18:55:30.243614 139620787287872 deprecation.py:323] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [21]:
# # image_name_train
# # senti_train
# # cap_vector_train
# sess= tf.Session()
# init_op= tf.global_variables_initializer()
# with sess.as_default():
#     init_op.run()
#     batch_size= BATCH_SIZE
#     start=0
#     for i in tqdm(range(10)):
#         input_imgs=[]
#         for j in range(batch_size):
#             img= load_image( image_name_train[start+j])
#             input_imgs.append(img)
#         input_imgs= np.array(input_imgs)
#         input_sentis= senti_train[start:start+batch_size]
#         input_sequences= cap_vector_train[start:start+batch_size]
#         start+=batch_size
#             
# 
#         train_step.run (feed_dict={image_input:input_imgs,seq_input: input_sequences, senti_input: input_sentis})
#         

In [44]:
def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = keras.losses.sparse_categorical_crossentropy(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)


In [14]:
image_input=Input(shape=(299,299,3),dtype=tf.float32)
seq_input= Input(shape=(max_length,),dtype=tf.int32)
senti_input= Input(shape=(1,),dtype=tf.int32)

image_feature=image_model(image_input) 
image_feature= Flatten()(image_feature)
embedded_image= keras.layers.Dense(embedding_size)(image_feature)

embedded_seq= Embedding(input_dim=1,output_dim=units)(seq_input)

func2= Lambda (lambda w: tf.expand_dims(w, 1))
expanded_image=func2(embedded_image)

x= Concatenate(axis=1)([expanded_image, embedded_seq])
# print(x.shape)
x= Reshape(( max_length+1,units))(x)
output = keras.layers.LSTM(units, return_sequences=True,
                           return_state=False,
                           recurrent_initializer='glorot_uniform')(x)


# lstm_output= tf.reshape(output,(-1,units))
lstm_output= Reshape((-1,units))(output)
logits= keras.layers.Dense(vocab_size,activation='softmax')(lstm_output)
func3= Lambda (lambda w: tf.slice(w,[0,0,0],[-1,max_length,-1]))
logits_2= func3(logits)

# target=seq_input[:,0:max_length]
# 
# loss= tf.reduce_mean(keras.losses.sparse_categorical_crossentropy(target,logits))
# 
# mask = tf.math.logical_not(tf.math.equal(target, 0))
# mask = tf.cast(mask, dtype=loss.dtype)
# batch_loss= loss * mask
# 
# train_step= tf.train.AdamOptimizer().minimize(batch_loss)
# inference_step= tf.argmax(logits,axis=-1)



In [15]:
new_model= keras.Model([image_input,seq_input,senti_input],logits_2)
new_model.compile(loss='sparse_categorical_crossentropy',optimizer='Adam')

print(new_model.output_shape)

W0921 00:15:02.001816 139620787287872 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



(None, 10, 500)


In [17]:
# for i in tqdm(range(1)):
input_imgs=[]
for j in range(1000):
    img= load_image( image_name_train[start+j])
    input_imgs.append(img)
input_imgs= np.array(input_imgs)
input_sentis= np.array(senti_train[:1000])
input_sequences= np.array(cap_vector_train[:1000])
target= np.expand_dims(input_sequences,axis=-1)
print(input_imgs.shape)
print(input_sequences.shape)
print(input_sentis.shape)
new_model.fit(x=[input_imgs,input_sequences,input_sentis],y=target, batch_size=1,verbose=1,epochs=1)


W0921 00:19:56.354986 139620787287872 deprecation.py:323] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


(1000, 299, 299, 3)
(1000, 10)
(1000,)


Epoch 1/1


UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node model_1/conv2d_1/convolution}}]]
	 [[loss/mul/_3887]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node model_1/conv2d_1/convolution}}]]
0 successful operations.
0 derived errors ignored.

In [ ]:
new_model.fit(x=input_x,y=input_y,batch_size=BATCH_SIZE,verbose=1,epochs=1)

In [ ]:
class RNN_Decoder(keras.Model):
    def __init__(self, embedding_size, units, vocab_size):
        super(RNN_Decoder, self).__init__()
        self.img_embedding_size= embedding_size
        
        self.vocab_size= vocab_size
    
        self.units = units
        #seq emnedding
        self.embedding = keras.layers.Embedding(vocab_size, embedding_size)
        
        self.lstm = keras.layers.LSTM(self.units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        
        self.cnn_fc1 = keras.layers.Dense(self.img_embedding_size)
        
        self.lstm_fc1 = keras.layers.Dense(self.units)
        self.lstm_fc2 = keras.layers.Dense(self.vocab_size) 
        
    def call(self, image_input, seq_input, senti_input):
        embedded_image= self.cnn_fc1(image_input)
        
        embedded_seq= self.lstm_fc1(seq_input)
        
        x= tf.concat([tf.expand_dims(embedded_image, 1), embedded_seq], axis=1)
        
        output, state= self.lstm(x)
        lstm_output= tf.reshape(output,(-1,self.units))
        
        
        logits= self.lstm_fc2(lstm_output)
        return logits, state 



